# Fitbit Exploration

In [109]:
import os
import sys
sys.path.append('../')

from src.features import build_features
from src.visualization import visualize
from src.reports import make_report

import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np

from datetime import datetime, timedelta
import json
import ast

# Data Import

## Functions
Below are a set of useful functions for handling the Fitbit Data

In [110]:
def get_device_df(info_df):
    '''
    Take dictionary-like entries for fitbit info dataframe for each row in a dataframe and makes a new dataframe
    
    Inputs:
    - info_df: the fitbit info dataframe with the dictionary-like entries
    
    Returns a dataframe for the device column
    '''
    
    overall_dict = {}
    for row in range(len(info_df)):
        Dict = ast.literal_eval(info_df['devices'][row])
        if type(Dict) == dict:
            Dict = Dict
        elif type(Dict) in [tuple,list] and len(Dict) > 1:
            Dict = Dict[0]
        else:
            continue

        for key in Dict.keys():
            overall_dict.setdefault(key, [])
            overall_dict[key].append(Dict[key])
        # adding in the date of recording
        overall_dict.setdefault('date', [])
        overall_dict['date'].append(info_df.index[row])
        
    df = pd.DataFrame(overall_dict)
    df['date'] = pd.to_datetime(df['date'],errors='coerce')
    return df.set_index('date')

In [111]:
def process_fitbit_intraday(raw_df,resample_rate=60):
    '''
    
    '''
    try:
        df = raw_df.resample(f'{resample_rate}T').mean()
    except TypeError:
        print(f"\t\tDataframe is most likely empty ({len(raw_df)})")
        return raw_df
    return df

## Participant Info

In [112]:
id_crossover = pd.read_excel(f'../data/raw/utx000/admin/id_crossover.xlsx',sheet_name='all')
beacon_id = pd.read_excel(f'../data/raw/utx000/admin/id_crossover.xlsx',sheet_name='beacon')

## Single Participant
Starting with one participant to see how to work with the data

In [113]:
pt = "2xtqkfz1"
#pt_dir = f"/Volumes/HEF_Dissertation_Research/utx000/extension/data/fitbit/{pt}"
pt_dir = f"../data/raw/utx000/fitbit/{pt}"
single_info = pd.read_csv(f"{pt_dir}/fitbit/fitbit_info.csv")
single_daily = pd.read_csv(f"{pt_dir}/fitbit/fitbit_daily_records.csv")
single_intra = pd.read_csv(f"{pt_dir}/fitbit/fitbit_intraday_records.csv",index_col=0,parse_dates=True)

### Info CSV
Below is an output of the ```fitbit_info.csv```

In [114]:
single_info.head()

,date,devices,friends,friends_leaderboard
0,2020-10-02 22:00:08.291453,"[{'battery': 'High', 'batteryLevel': 93, 'devi...",[],"[{'id': '87GWSF', 'type': 'inactive-user'}]"


<div class="alert alert-block alert-danger"> 
    The get_device_df function does not work, but it doesn't give us any good information anyway so I will leave it unchanged

In [115]:
#devices = get_device_df(single_info)
#devices.head()

The ```fitbit_info.csv``` seems to have good information for debugging purposes and tracking battery levels, but overall won't be useful for any analysis purposes.

### Daily CSV

In [116]:
single_daily.head()

,date,activities_calories,activities_caloriesBMR,activities_steps,activities_distance,activities_minutesSedentary,activities_minutesLightlyActive,activities_minutesFairlyActive,activities_minutesVeryActive,activities_activityCalories,body_bmi,body_fat,body_weight,foods_log_caloriesIn,foods_log_water,activities_heart,sleep
0,2020-05-01 00:00:00,1320.0,1320.0,0,0.0,1440,0,0,0,0.0,19.993023,0.0,119.99,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN
1,2020-05-02 00:00:00,1320.0,1320.0,0,0.0,1440,0,0,0,0.0,19.993023,0.0,119.99,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN
2,2020-05-03 00:00:00,1320.0,1320.0,0,0.0,1440,0,0,0,0.0,19.993023,0.0,119.99,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN
3,2020-05-04 00:00:00,1320.0,1320.0,0,0.0,1440,0,0,0,0.0,19.993023,0.0,119.99,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN
4,2020-05-05 00:00:00,1320.0,1320.0,0,0.0,1440,0,0,0,0.0,19.993023,0.0,119.99,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN


In [117]:
single_daily["sleep"].unique()

array([nan,
       "[{'dateOfSleep': '2020-07-10', 'duration': 19740000, 'efficiency': 91, 'endTime': '2020-07-10T09:11:00.000', 'infoCode': 0, 'isMainSleep': True, 'levels': {'data': [{'dateTime': '2020-07-10T03:42:00.000', 'level': 'wake', 'seconds': 720}, {'dateTime': '2020-07-10T03:54:00.000', 'level': 'light', 'seconds': 960}, {'dateTime': '2020-07-10T04:10:00.000', 'level': 'deep', 'seconds': 2550}, {'dateTime': '2020-07-10T04:52:30.000', 'level': 'light', 'seconds': 2880}, {'dateTime': '2020-07-10T05:40:30.000', 'level': 'wake', 'seconds': 240}, {'dateTime': '2020-07-10T05:44:30.000', 'level': 'light', 'seconds': 1860}, {'dateTime': '2020-07-10T06:15:30.000', 'level': 'deep', 'seconds': 600}, {'dateTime': '2020-07-10T06:25:30.000', 'level': 'light', 'seconds': 1170}, {'dateTime': '2020-07-10T06:45:00.000', 'level': 'wake', 'seconds': 1050}, {'dateTime': '2020-07-10T07:02:30.000', 'level': 'light', 'seconds': 30}, {'dateTime': '2020-07-10T07:03:00.000', 'level': 'rem', 'seconds':

### Intraday CSV

In [118]:
single_intra.head()

,activities_calories,activities_steps,activities_distance,activities_heart
date,,,,
2020-05-20 14:57:29,NaN,NaN,NaN,70.0
2020-05-20 14:58:00,1.0087,NaN,NaN,NaN
2020-05-20 14:58:28,NaN,NaN,NaN,70.0
2020-05-20 14:58:43,NaN,NaN,NaN,70.0
2020-05-20 14:58:58,NaN,NaN,NaN,70.0


## All Participants
Now that we have looked at the individual data files, now we can start to import all the data for all participants.

In [119]:
def import_fitbit(filename, pt_dir=f"/Volumes/HEF_Dissertation_Research/utx000/extension/data/fitbit/",verbose=False):
    '''
    
    '''
    print(f"\tReading from file {filename}")
    df = pd.DataFrame()
    for pt in os.listdir(pt_dir):
        if pt[0] != ".":
            if verbose:
                print(f"\t\tReading for participant {pt}")
            try:
                temp = pd.read_csv(f"{pt_dir}{pt}/fitbit/fitbit_{filename}.csv", index_col=0, parse_dates=True)
                if filename[:4] == "intr":
                    temp = process_fitbit_intraday(temp)
                
                temp["beiwe"] = pt
                df = df.append(temp)
            except FileNotFoundError:
                print(f"\t\tFile {filename} not found for participant {pt}")
                
    return df

In [120]:
#fitbit_info = import_fitbit("info")
#fitbit_intra = import_fitbit("intraday_records")

### Daily Fitbit
Below is an output of the ```daily_fitbit.csv``` file and a closer look at some of the more interesting entries.

In [121]:
pt_dir = f"../data/raw/utx000/fitbit/"
fitbit_daily = import_fitbit("daily_records",pt_dir)
fitbit_daily.head()

	Reading from file daily_records
		File daily_records not found for participant 789gcb6u


,activities_calories,activities_caloriesBMR,activities_steps,activities_distance,activities_minutesSedentary,activities_minutesLightlyActive,activities_minutesFairlyActive,activities_minutesVeryActive,activities_activityCalories,body_bmi,body_fat,body_weight,foods_log_caloriesIn,foods_log_water,activities_heart,sleep,beiwe
date,,,,,,,,,,,,,,,,,
2020-05-01,1881.0,1881.0,0,0.0,1440,0,0,0,0.0,23.754,0.0,180.0,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN,hfttkth7
2020-05-02,1881.0,1881.0,0,0.0,1440,0,0,0,0.0,23.754,0.0,180.0,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN,hfttkth7
2020-05-03,1881.0,1881.0,0,0.0,1440,0,0,0,0.0,23.754,0.0,180.0,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN,hfttkth7
2020-05-04,1881.0,1881.0,0,0.0,1440,0,0,0,0.0,23.754,0.0,180.0,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN,hfttkth7
2020-05-05,1881.0,1881.0,0,0.0,1440,0,0,0,0.0,23.754,0.0,180.0,0.0,0.0,"{'customHeartRateZones': [], 'heartRateZones':...",NaN,hfttkth7


In [122]:
fitbit_daily[fitbit_daily["beiwe"] == "hfttkth7"].dropna().head()["sleep"][1]

"[{'dateOfSleep': '2020-05-15', 'duration': 29580000, 'efficiency': 87, 'endTime': '2020-05-15T08:06:30.000', 'infoCode': 0, 'isMainSleep': True, 'levels': {'data': [{'dateTime': '2020-05-14T23:53:30.000', 'level': 'wake', 'seconds': 660}, {'dateTime': '2020-05-15T00:04:30.000', 'level': 'light', 'seconds': 540}, {'dateTime': '2020-05-15T00:13:30.000', 'level': 'deep', 'seconds': 840}, {'dateTime': '2020-05-15T00:27:30.000', 'level': 'light', 'seconds': 480}, {'dateTime': '2020-05-15T00:35:30.000', 'level': 'deep', 'seconds': 360}, {'dateTime': '2020-05-15T00:41:30.000', 'level': 'light', 'seconds': 2670}, {'dateTime': '2020-05-15T01:26:00.000', 'level': 'deep', 'seconds': 2820}, {'dateTime': '2020-05-15T02:13:00.000', 'level': 'light', 'seconds': 1020}, {'dateTime': '2020-05-15T02:30:00.000', 'level': 'wake', 'seconds': 360}, {'dateTime': '2020-05-15T02:36:00.000', 'level': 'light', 'seconds': 570}, {'dateTime': '2020-05-15T02:45:30.000', 'level': 'wake', 'seconds': 240}, {'dateTime':

In [123]:
fitbit_daily[fitbit_daily["beiwe"] == "awa8uces"].dropna()["sleep"][0]

"[{'awakeCount': 1, 'awakeDuration': 1, 'awakeningsCount': 6, 'dateOfSleep': '2020-05-18', 'duration': 6120000, 'efficiency': 81, 'endTime': '2020-05-18T18:13:30.000', 'logId': 27234597145, 'minuteData': [{'dateTime': '16:31:00', 'value': '2'}, {'dateTime': '16:32:00', 'value': '2'}, {'dateTime': '16:33:00', 'value': '1'}, {'dateTime': '16:34:00', 'value': '1'}, {'dateTime': '16:35:00', 'value': '1'}, {'dateTime': '16:36:00', 'value': '1'}, {'dateTime': '16:37:00', 'value': '1'}, {'dateTime': '16:38:00', 'value': '1'}, {'dateTime': '16:39:00', 'value': '1'}, {'dateTime': '16:40:00', 'value': '1'}, {'dateTime': '16:41:00', 'value': '1'}, {'dateTime': '16:42:00', 'value': '1'}, {'dateTime': '16:43:00', 'value': '1'}, {'dateTime': '16:44:00', 'value': '1'}, {'dateTime': '16:45:00', 'value': '1'}, {'dateTime': '16:46:00', 'value': '1'}, {'dateTime': '16:47:00', 'value': '1'}, {'dateTime': '16:48:00', 'value': '1'}, {'dateTime': '16:49:00', 'value': '1'}, {'dateTime': '16:50:00', 'value': '

The ```fitbit_daily.csv``` has a lot of good summary data including sleep summary values and sleep data by the minute.

In [124]:
def get_daily_sleep(daily_df,verbose=False):
    '''
    Creates a dataframe with the daily sleep data summarized

    Inputs:
    - daily_df: dataframe created from the daily fitbit csv file

    Returns a dataframe of the daily sleep data
    '''
    overall_dict = {"dateOfSleep": [],"duration": [],"efficiency":[],"endTime": [],"infoCode": [],"isMainSleep": [],"levels": [],"logId": [],
            "minutesAfterWakeup": [],"minutesAsleep": [],"minutesAwake": [],"minutesToFallAsleep": [],"startTime": [],"timeInBed": [],
            "type": [],"date": [],"beiwe": [],"awakeCount": [],"awakeDuration": [],"awakeningsCount": [],"minuteData": [],"restlessCount": [],"restlessDuration": []}
    for row in range(len(daily_df)):
        # in case Fitbit didn't record sleep records for that night - value is NaN
        pt = daily_df['beiwe'][row]
        if verbose:
            print(f"\t\tWorking for Participant {pt}")
        if type(daily_df['sleep'][row]) == float:
            continue
        else:
            Dict = ast.literal_eval(daily_df['sleep'][row])
            if type(Dict) == dict:
                Dict = Dict
            else:
                Dict = Dict[0]
            for key in overall_dict.keys():
                #overall_dict.setdefault(key, [])
                if key in ["date","beiwe","redcap","beacon"]:
                    pass
                elif key in Dict.keys():
                    overall_dict[key].append(Dict[key])
                else:
                    overall_dict[key].append(np.nan)
            # adding in the date of recording
            overall_dict.setdefault('date', [])
            overall_dict['date'].append(daily_df.index[row])
            # adding beiwe id
            bid = daily_df['beiwe'][row]
            overall_dict.setdefault('beiwe', [])
            overall_dict['beiwe'].append(bid)
            # adding other ids
            crossover_info = id_crossover.loc[id_crossover['beiwe']==bid].reset_index(drop=True)
            try:
                bb = crossover_info['beacon'].iloc[0]
            except IndexError:
                bb = np.nan
            try:
                rid = crossover_info['redcap'].iloc[0]
            except IndexError:
                rid = np.nan
            del crossover_info
            overall_dict.setdefault('redcap', [])
            overall_dict['redcap'].append(rid)
            overall_dict.setdefault('beacon', [])
            overall_dict['beacon'].append(bb)

    #for key in overall_dict.keys():
        #print(f"{key}: {len(overall_dict[key])}")
    df = pd.DataFrame(overall_dict)
    df['date'] = pd.to_datetime(df['date'],errors='coerce')
    # removing classic sleep stage data
    df = df[df['type'] != 'classic']
    # dropping/renaming columns
    df.drop(["dateOfSleep","infoCode","logId","type","awakeCount","awakeDuration","awakeningsCount","minuteData","restlessCount","restlessDuration"],axis=1,inplace=True)
    df.columns = ["duration_ms","efficiency","end_time","main_sleep","levels","minutes_after_wakeup","minutes_asleep","minutes_awake","minutes_to_sleep","start_time","time_in_bed","date","beiwe","redcap","beacon"]
    # main sleep issues
    #df["main_sleep"].fillna(True,inplace=True)
    # recalculating se because Fitbit determines it some unknown/incorrect way
    df["efficiency"] = df["minutes_asleep"] / df["time_in_bed"] * 100
    return df

In [125]:
daily_slp = get_daily_sleep(fitbit_daily,verbose=False)
daily_slp.head()

,duration_ms,efficiency,end_time,main_sleep,levels,minutes_after_wakeup,minutes_asleep,minutes_awake,minutes_to_sleep,start_time,time_in_bed,date,beiwe,redcap,beacon
0,24360000,93.349754,2020-05-14T07:13:00.000,True,{'data': [{'dateTime': '2020-05-14T00:27:00.00...,0,379,27,0,2020-05-14T00:27:00.000,406,2020-05-14,hfttkth7,NaN,NaN
1,29580000,79.513185,2020-05-15T08:06:30.000,True,{'data': [{'dateTime': '2020-05-14T23:53:30.00...,8,392,101,0,2020-05-14T23:53:30.000,493,2020-05-15,hfttkth7,NaN,NaN
2,19740000,87.234043,2020-05-16T04:57:00.000,True,{'data': [{'dateTime': '2020-05-15T23:28:00.00...,7,287,42,0,2020-05-15T23:28:00.000,329,2020-05-16,hfttkth7,NaN,NaN
3,26820000,90.156600,2020-05-17T09:28:30.000,True,{'data': [{'dateTime': '2020-05-17T02:01:30.00...,8,403,44,0,2020-05-17T02:01:30.000,447,2020-05-17,hfttkth7,NaN,NaN
4,24960000,84.375000,2020-05-18T07:20:00.000,True,{'data': [{'dateTime': '2020-05-18T00:24:00.00...,0,351,65,0,2020-05-18T00:24:00.000,416,2020-05-18,hfttkth7,NaN,NaN


In [126]:
daily_slp[daily_slp["beiwe"] == "awa8uces"].head()

,duration_ms,efficiency,end_time,main_sleep,levels,minutes_after_wakeup,minutes_asleep,minutes_awake,minutes_to_sleep,start_time,time_in_bed,date,beiwe,redcap,beacon
2380,6120000,81.372549,2020-05-18T18:13:30.000,NaN,NaN,0,83,19,0,2020-05-18T16:31:00.000,102,2020-05-18,awa8uces,28.0,26.0
2381,4500000,89.333333,2020-05-19T18:58:30.000,NaN,NaN,0,67,8,0,2020-05-19T17:43:00.000,75,2020-05-19,awa8uces,28.0,26.0
2382,5220000,91.954023,2020-05-20T17:01:00.000,NaN,NaN,0,80,7,0,2020-05-20T15:33:30.000,87,2020-05-20,awa8uces,28.0,26.0
2383,28380000,95.771670,2020-05-21T12:57:30.000,NaN,NaN,0,453,20,0,2020-05-21T05:04:00.000,473,2020-05-21,awa8uces,28.0,26.0
2384,4860000,95.061728,2020-05-22T14:03:00.000,NaN,NaN,0,77,4,0,2020-05-22T12:42:00.000,81,2020-05-22,awa8uces,28.0,26.0


In [127]:
daily_slp['main_sleep'] = daily_slp.apply(
    lambda row: True if np.isnan(row['main_sleep']) and row["minutes_asleep"] > 180 else row['main_sleep'],
    axis=1
)

In [128]:
def get_sleep_stages(daily_sleep):
    '''
    Creates a dataframe for the minute sleep data

    Input(s):
    - daily_sleep: dataframe holding the daily sleep data with a column called minuteData

    Returns:
    - sleep_stages: a dataframe with sleep stage data for every stage transition
    - summary: a dataframe with the nightly sleep stage information
    '''

    data_dict = {'startDate':[],'endDate':[],'dateTime':[],'level':[],'seconds':[],'beiwe':[]}
    summary_dict = {'start_date':[],'end_date':[],'deep_count':[],'deep_minutes':[],'light_count':[],'light_minutes':[],
                    'rem_count':[],'rem_minutes':[],'wake_count':[],'wake_minutes':[],'beiwe':[]}
    for i in range(len(daily_sleep)):
        d0 = pd.to_datetime(daily_sleep.iloc[i,:]["start_time"])
        d1 = pd.to_datetime(daily_sleep.iloc[i,:]["date"])
        try:
            sleep_dict = daily_sleep.iloc[i,:]["levels"]
            for key in sleep_dict.keys():
                if key == 'data': # data without short wake periods
                    temp_data = sleep_dict['data']
                    for temp_data_dict in temp_data:
                        for data_key in temp_data_dict.keys():
                            data_dict[data_key].append(temp_data_dict[data_key])
                        data_dict['startDate'].append(d0.date())
                        data_dict['endDate'].append(d1.date())
                        data_dict['beiwe'].append(daily_sleep.iloc[i,:]['beiwe'])
                elif key == 'summary': # nightly summary data - already in dictionary form
                    for summary_key in sleep_dict['summary'].keys():
                        stage_dict = sleep_dict['summary'][summary_key]
                        for stage_key in ['count','minutes']:
                            summary_dict[f'{summary_key}_{stage_key}'].append(stage_dict[stage_key])

                    summary_dict['start_date'].append(d0.date())
                    summary_dict['end_date'].append(d1.date())
                    summary_dict['beiwe'].append(daily_sleep.iloc[i,:]['beiwe'])
                else: # shortData or data with short wake periods - don't need
                    pass
        except AttributeError:
            pass

    sleep_stages = pd.DataFrame(data_dict)
    sleep_stages.columns = ['start_date','end_date','time','stage','time_at_stage','beiwe'] # renaming columns
    # adding column for numeric value of sleep stage 
    def numeric_from_str_sleep_stage(row):
        if row['stage'] == 'wake':
            return 0
        elif row['stage'] == 'light':
            return 1
        elif row['stage'] == 'deep':
            return 2
        elif row['stage'] == 'rem':
            return 3
        else:
            return np.nan

    sleep_stages['value'] = sleep_stages.apply(lambda row: numeric_from_str_sleep_stage(row), axis=1)

    summary = pd.DataFrame(summary_dict)
    # getting sol
    sol = sleep_stages.groupby(["beiwe","start_date"]).first().reset_index()
    sol = sol[sol["stage"] == "wake"]
    sol["sol"] = sol["time_at_stage"] / 60
    # getting wol
    wol = sleep_stages.groupby(["beiwe","start_date"]).last().reset_index()
    wol = wol[wol["stage"] == "wake"]
    wol["wol"] = wol["time_at_stage"] / 60
    wol["date"] = pd.to_datetime(wol["end_date"],errors="coerce")
    return sleep_stages, summary, sol[["beiwe","time","sol"]], wol[["beiwe","date","wol"]]

In [129]:
stages, stages_summary, sol, wol = get_sleep_stages(daily_slp)
daily_slp.drop(["levels"],axis=1,inplace=True)
# adding SOL to daily sleep from sleep stages
daily_slp = daily_slp.merge(right=sol,left_on=["beiwe","start_time"],right_on=["beiwe","time"],how="left")
daily_slp["sol"].fillna(0,inplace=True)
# adding WOL to daily sleep from sleep stages
daily_slp["date"] = pd.to_datetime(daily_slp["date"],errors="coerce")
daily_slp = daily_slp.merge(right=wol,on=["beiwe","date"],how="left")
daily_slp["wol"].fillna(0,inplace=True)
daily_slp.drop("time",axis="columns",inplace=True)

In [130]:
stages[stages["beiwe"] == "awa8uces"]

,start_date,end_date,time,stage,time_at_stage,beiwe,value


In [141]:
def get_fitbit_sleep_summary():
    '''
    Gets summary sleep data from Fitbit including sleep stages and daily sleep summaries

    Returns a dataframes pertaining to the sleep metrics from Fitbit
    '''
    # Fitbit Summary and Sleep Stages Summary
    # ---------------------------------------
    print("\tCombining Fitbit Sleep Measurements")
    # sleep summary
    daily_slp["date"] = pd.to_datetime(daily_slp["date"])
    # sleep stages
    stages_summary["end_date"] = pd.to_datetime(stages_summary["end_date"])
    # combining
    fb_all = stages_summary.merge(daily_slp,left_on=["end_date","beiwe"],right_on=["date","beiwe"],how="right")
    # filtering data
    fb_all = fb_all[fb_all["main_sleep"] == True]
    # creating features
    for sleep_stage_metric in ["count","minutes"]:
        fb_all[f"nrem_{sleep_stage_metric}"] = fb_all[f"light_{sleep_stage_metric}"] + fb_all[f"deep_{sleep_stage_metric}"] # nrem count and minutes

    fb_all[f"rem2nrem"] = fb_all[f"rem_minutes"] / fb_all[f"nrem_minutes"] # ratio of nrem to rem (count and minutes)

    fb_all["tst_fb"] = fb_all["minutes_asleep"] / 60
    for stage in ["rem","nrem"]:
        fb_all[f"{stage}_percent"] = fb_all[f"{stage}_minutes"] / (fb_all["tst_fb"]*60)
    fb_all["waso"] = fb_all["minutes_awake"] - fb_all["sol"] - fb_all["wol"]
    fb_all["sol_fb"] = fb_all["sol"]
    fb_all["wol_fb"] = fb_all["wol"]
    fb_all.drop(["sol","wol"],axis="columns",inplace=True)
    # dropping unecessary columns
    fb_all.drop(["main_sleep","duration_ms","minutes_after_wakeup","time_in_bed","minutes_asleep","minutes_to_sleep","minutes_awake"],axis="columns",inplace=True)
    print(fb_all[fb_all["beiwe"] == "awa8uces"])
    fb_all[f"start_date"] = fb_all.apply(
            lambda row: row[f'start_time'].date if pd.isnull(row[f'start_date']) else row[f'start_date'],
            axis=1)

    return fb_all

In [142]:
summary = get_fitbit_sleep_summary()
summary[summary["beiwe"] == "awa8uces"]

	Combining Fitbit Sleep Measurements
     start_date end_date  deep_count  deep_minutes  light_count  \
2024        NaN      NaT         NaN           NaN          NaN   
2027        NaN      NaT         NaN           NaN          NaN   
2029        NaN      NaT         NaN           NaN          NaN   
2031        NaN      NaT         NaN           NaN          NaN   
2032        NaN      NaT         NaN           NaN          NaN   
2033        NaN      NaT         NaN           NaN          NaN   
2035        NaN      NaT         NaN           NaN          NaN   
2036        NaN      NaT         NaN           NaN          NaN   
2038        NaN      NaT         NaN           NaN          NaN   
2039        NaN      NaT         NaN           NaN          NaN   
2040        NaN      NaT         NaN           NaN          NaN   
2041        NaN      NaT         NaN           NaN          NaN   
2042        NaN      NaT         NaN           NaN          NaN   
2044        NaN      NaT 

AttributeError: 'str' object has no attribute 'dt'